In [1]:
cd ../..

D:\Users\mrefoyo\Proyectos\Sub-SpaCE_plus


In [2]:
import numpy as np
import pandas as pd

from experiments.experiment_utils import local_data_loader

In [5]:
models = ['cls_basic_train']

univariate_datasets = [
    'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'Car', 'CBF',
    'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers',
    'DiatomSizeReduction',
    'DistalPhalanxOutlineCorrect', 'DistalPhalanxOutlineAgeGroup',
    'DistalPhalanxTW', 'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays',
    'ElectricDevices', 'FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish',
    'FordA', 'FordB', 'GunPoint', 'Ham', 'HandOutlines', 'Haptics', 'Herring',
    'InlineSkate', 'InsectWingbeatSound', 'ItalyPowerDemand',
    'LargeKitchenAppliances', 'Lightning2', 'Lightning7', 'Mallat', 'Meat',
    'MedicalImages', 'MiddlePhalanxOutlineCorrect',
    'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxTW', 'MoteStrain',
    'NonInvasiveFatalECGThorax1', 'NonInvasiveFatalECGThorax2', 'OliveOil',
    'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 'Plane',
    'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxOutlineAgeGroup',
    'ProximalPhalanxTW', 'RefrigerationDevices', 'ScreenType', 'ShapeletSim',
    'ShapesAll', 'SmallKitchenAppliances', 'SonyAIBORobotSurface1',
    'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 'SwedishLeaf',
    'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2',
    'Trace', 'TwoLeadECG', 'TwoPatterns',
    'Wafer', 'Wine', 'WordSynonyms', 'Worms', 'WormsTwoClass', 'Yoga'
]
univariate_datasets = [
    'ECG200', 'Gunpoint', 'Coffee',
    'ItalyPowerDemand', 'ProximalPhalanxOutlineCorrect', 'Strawberry', 'FordA', 'HandOutlines',
    'Plane', 'TwoPatterns', 'FacesUCR', 'ECG5000', 'CinCECGTorso', 'NonInvasiveFatalECGThorax2', 'CBF',
]


multivariate_datasets = [
    'BasicMotions', 'NATOPS', 'UWaveGestureLibrary',
    'ArticularyWordRecognition', 'AtrialFibrillation', 'CharacterTrajectories', 'Cricket',
    'DuckDuckGeese', 'EigenWorms', 'Epilepsy', 'EthanolConcentration', 'ERing',  'FaceDetection',
    'FingerMovements', 'HandMovementDirection', 'Handwriting', 'Heartbeat', 'InsectWingbeat', 'JapaneseVowels',
    'Libras', 'LSST', 'MotorImagery', 'PenDigits', 'PEMS-SF', 'Phoneme', 'RacketSports',
    'SelfRegulationSCP1', 'SelfRegulationSCP2', 'SpokenArabicDigits', 'StandWalkJump'
]

datasets = univariate_datasets + multivariate_datasets

In [31]:
best_results_df = pd.DataFrame()
for dataset in datasets:
    for model in models:
        try:
            # Obtain results of dataset
            dataset_results_df = pd.read_excel(f'./experiments/models/{dataset}/{model}/all_combination_results.xlsx', index_col=0)
            dataset_results_df.insert(0, "Dataset", dataset)
            dataset_results_df = dataset_results_df.rename(columns={"model_type": "Model Type", "test_f1": "F1-score"})
            best_dataset_results_df = dataset_results_df[['Dataset', 'Model Type', 'F1-score']].iloc[0:1]

            
            # Obtain dataset information
            X_train, y_train, X_test, y_test = local_data_loader(str(dataset), False, data_path="./experiments/data")
            if X_train.shape[2] > 1:
                best_dataset_results_df.insert(1,'Archive', "UEA")
            else:
                best_dataset_results_df.insert(1,'Archive', "UCR")
            best_dataset_results_df.insert(1,'Classes', len(np.unique(y_train)))
            best_dataset_results_df.insert(1,'Channels', X_train.shape[2])
            best_dataset_results_df.insert(1, 'Length', X_train.shape[1])
            best_dataset_results_df.insert(1, 'Test size', X_test.shape[0])
            best_dataset_results_df.insert(1, 'Train size', X_train.shape[0])

            # Concat results
            best_results_df = pd.concat([best_results_df, best_dataset_results_df])
        except FileNotFoundError as msg:
            print(f'Problem with dataset {dataset}: {msg}')

# Filter and sort results df
best_results_df = best_results_df.sort_values(by='F1-score', ascending=False).reset_index(drop=True)


Problem with dataset AtrialFibrillation: [Errno 2] No such file or directory: './experiments/models/AtrialFibrillation/cls_basic_train/all_combination_results.xlsx'
Problem with dataset CharacterTrajectories: [Errno 2] No such file or directory: './experiments/models/CharacterTrajectories/cls_basic_train/all_combination_results.xlsx'
Problem with dataset DuckDuckGeese: [Errno 2] No such file or directory: './experiments/models/DuckDuckGeese/cls_basic_train/all_combination_results.xlsx'
Problem with dataset EigenWorms: [Errno 2] No such file or directory: './experiments/models/EigenWorms/cls_basic_train/all_combination_results.xlsx'
Problem with dataset ERing: [Errno 2] No such file or directory: './experiments/models/ERing/cls_basic_train/all_combination_results.xlsx'
Problem with dataset InsectWingbeat: [Errno 2] No such file or directory: './experiments/models/InsectWingbeat/cls_basic_train/all_combination_results.xlsx'
Problem with dataset JapaneseVowels: [Errno 2] No such file or d

In [33]:
aux = best_results_df.sort_values(by=["Archive", "F1-score"], ascending=[True, False]).head(30)
aux = aux.set_index("Dataset").drop("Archive", axis=1)
aux = aux[aux["F1-score"] > 0.8]
aux = aux.style.format(decimal='.', precision=3)
aux

,Train size,Test size,Length,Channels,Classes,Model Type,F1-score
Dataset,,,,,,,
TwoPatterns,1000,4000,128,1,4,cnn-complex,1.000
Strawberry,613,370,235,1,2,cnn-complex,0.981
Plane,105,105,144,1,7,cnn-complex,0.981
ItalyPowerDemand,67,1029,24,1,2,cnn-simple,0.960
CinCECGTorso,40,1380,1639,1,4,cnn-simple,0.951
Gunpoint,50,150,150,1,2,cnn-complex,0.946
Coffee,28,28,286,1,2,cnn-complex,0.928
HandOutlines,1000,370,2709,1,2,cnn-intermediate,0.920
FordA,3601,1320,500,1,2,cnn-complex,0.918


In [37]:
print(aux.to_latex())

\begin{tabular}{lrrrrrlr}
 & Train size & Test size & Length & Channels & Classes & Model Type & F1-score \\
Dataset &  &  &  &  &  &  &  \\
TwoPatterns & 1000 & 4000 & 128 & 1 & 4 & cnn-complex & 1.000 \\
Strawberry & 613 & 370 & 235 & 1 & 2 & cnn-complex & 0.981 \\
Plane & 105 & 105 & 144 & 1 & 7 & cnn-complex & 0.981 \\
ItalyPowerDemand & 67 & 1029 & 24 & 1 & 2 & cnn-simple & 0.960 \\
CinCECGTorso & 40 & 1380 & 1639 & 1 & 4 & cnn-simple & 0.951 \\
Gunpoint & 50 & 150 & 150 & 1 & 2 & cnn-complex & 0.946 \\
Coffee & 28 & 28 & 286 & 1 & 2 & cnn-complex & 0.928 \\
HandOutlines & 1000 & 370 & 2709 & 1 & 2 & cnn-intermediate & 0.920 \\
FordA & 3601 & 1320 & 500 & 1 & 2 & cnn-complex & 0.918 \\
NonInvasiveFatalECGThorax2 & 1800 & 1965 & 750 & 1 & 42 & cnn-complex & 0.915 \\
ECG5000 & 500 & 4500 & 140 & 1 & 5 & cnn-simple & 0.899 \\
ProximalPhalanxOutlineCorrect & 600 & 291 & 80 & 1 & 2 & cnn-simple & 0.899 \\
CBF & 30 & 900 & 128 & 1 & 3 & cnn-simple & 0.897 \\
ECG200 & 100 & 100 & 96 & 1 

In [43]:
aux = best_results_df.copy()
aux = aux[aux["F1-score"] > 0.8]
aux = aux[aux["Archive"] == "UCR"]
aux = aux.reset_index(drop=True).sort_values(by=["Classes", "Dataset"], ascending=[True, True])
aux

,Dataset,Train size,Test size,Length,Channels,Classes,Archive,Model Type,F1-score
6,Coffee,28,28,286,1,2,UCR,cnn-complex,0.927827
13,ECG200,100,100,96,1,2,UCR,cnn-simple,0.852992
8,FordA,3601,1320,500,1,2,UCR,cnn-complex,0.918186
5,Gunpoint,50,150,150,1,2,UCR,cnn-complex,0.946476
7,HandOutlines,1000,370,2709,1,2,UCR,cnn-intermediate,0.919712
3,ItalyPowerDemand,67,1029,24,1,2,UCR,cnn-simple,0.960148
11,ProximalPhalanxOutlineCorrect,600,291,80,1,2,UCR,cnn-simple,0.898883
1,Strawberry,613,370,235,1,2,UCR,cnn-complex,0.981127
12,CBF,30,900,128,1,3,UCR,cnn-simple,0.896858
4,CinCECGTorso,40,1380,1639,1,4,UCR,cnn-simple,0.951422


In [44]:
aux["Dataset"].values

array(['Coffee', 'ECG200', 'FordA', 'Gunpoint', 'HandOutlines',
       'ItalyPowerDemand', 'ProximalPhalanxOutlineCorrect', 'Strawberry',
       'CBF', 'CinCECGTorso', 'TwoPatterns', 'ECG5000', 'Plane',
       'FacesUCR', 'NonInvasiveFatalECGThorax2'], dtype=object)